# Order of convergence of FVM schemes
Consider the linear transport equation:

$u_t + a u_x = 0$

with initial conditions:

$u(x,0) = \sin(4\pi x)$

The exact solution of the problem is $u(x,t) = \sin (4 \pi (x-t))$

Let's plot the exact solution at time = 10 (note that the exact solution returns back to the initial condition at time $t = k$ for all positive integer $k$).

In [1]:
using Plots

In [2]:
u_exact(x,t) = sin(4*pi*(x-t))

u_exact (generic function with 1 method)

In [3]:
plot(0:0.01:1, [u_exact(x,10.0) for x in 0:0.01:1])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
<polyline clip-path="url(#clip9802)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.7332,193.812 51.079,170.956 56.4249,148.461 61.7707,126.681 67.1165,105.959 72.4623,86.6231 77.8081,68.9775 83.1539,53.3006 88.4998,39.8397 93.8456,28.807 
 99.1914,20.3766 104.537,14.6813 109.883,11.811 115.229,11.811 120.575,14.6813 125.92,20.3766 131.266,28.807 136.612,39.8397 141.958,53.3006 147.304,68.9775 
 152.65,86.6231 157.995,105.959 163.341,126.681 168.687,148.461 174.033,170.956 179.379,193.812 184.724,216.668 190.07,239.164 195.416,260.944 200.762,281.665 
 206.108,301.001 211.454,318.647 216.799,334.324 222.145,347.785 227.491,358.817 232.837,367.248 238.183,372.943 243.528,375.813 248.874,375.813 254.22,372.943 
 259.566,367.248 264.912,358.817 270.258,347.785 275.603,334.324 280.949,318.647 286.295,301.001 291.641,281.665 296.987,260.944 302.332,239.164 307.678,216.668 
 313.024,193.812 318.37,170.956 323.716,148.461 329.062,126.681 334.407,105.959 339.753,86.6231 345.099,68.9775 350.445,53.3006 355.791,39.8397 361.136,28.807 
 366.482,20.3766 371.828,14.6813 377.174,11.811 382.52,11.811 387.866,14.6813 393.211,20.3766 398.557,28.807 403.903,39.8397 409.249,53.3006 414.595,68.9775 
 419.94,86.6231 425.286,105.959 430.632,126.681 435.978,148.461 441.324,170.956 446.67,193.812 452.015,216.668 457.361,239.164 462.707,260.944 468.053,281.665 
 473.399,301.001 478.744,318.647 484.09,334.324 489.436,347.785 494.782,358.817 500.128,367.248 505.474,372.943 510.819,375.813 516.165,375.813 521.511,372.943 
 526.857,367.248 532.203,358.817 537.548,347.785 542.894,334.324 548.24,318.647 553.586,301.001 558.932,281.665 564.278,260.944 569.623,239.164 574.969,216.668 
 580.315,193.812 
 "/>
 
 
 
 
 y1

For the numerical solution, first we setup the problem:

In [4]:
using ConservationLawsDiffEq
# Define the flux and flux Jacobian
f(::Type{Val{:jac}},u::Vector) = eye(size(u,1))
f(u::Vector) = u

# Initial condition (using integral cell averages)
function u0_func(mesh)
  N = numcells(mesh)
  uinit = zeros(N, 1)
  faces = cell_faces(mesh)
  for i in 1:N
      uinit[i,1] = num_integrate(x->sin(4*pi*x),faces[i], faces[i+1])/volume(i, mesh)
  end
  return uinit
end

#Setup problem for a given N (number of cells/control volumenes on a uniform mesh)
#and given final time (Tend) with periodic boundary conditions
function get_problem(N, Tend; CFL = 0.5)
  mesh = Uniform1DFVMesh(N,0.0,1.0,:PERIODIC, :PERIODIC)
  u0 = u0_func(mesh)
  ConservationLawsProblem(u0,f,CFL,Tend,mesh)
end

get_problem (generic function with 1 method)

*Remark:* Note that in flux and flux jacobian definition we assume that $u$ is a vector even in the scalar case.

Now we approximate the solution using a first order Lax-Friedrichs scheme. 

In [5]:
prob = get_problem(50, 10.0)
sol1 = solve(prob, LaxFriedrichsAlgorithm();TimeIntegrator = Euler(), use_threads = true, save_everystep = false);

Let's compare the numerical solution against the exact solution:

In [6]:
plot(0:0.01:1, [u_exact(x,10.0) for x in 0:0.01:1], lab="u exact")
plot!(sol1, lab="u LF")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 x 
 
 
 u 
 
<polyline clip-path="url(#clip0002)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 61.4812,185.938 66.6696,164.071 71.8579,142.549 77.0463,121.711 82.2346,101.886 87.4229,83.3865 92.6113,66.5043 97.7996,51.5056 102.988,38.6271 108.176,28.0717 
 113.365,20.006 118.553,14.5571 123.741,11.811 128.93,11.811 134.118,14.5571 139.306,20.006 144.495,28.0717 149.683,38.6271 154.871,51.5056 160.06,66.5043 
 165.248,83.3865 170.436,101.886 175.625,121.711 180.813,142.549 186.001,164.071 191.19,185.938 196.378,207.805 201.566,229.328 206.755,250.165 211.943,269.99 
 217.131,288.49 222.32,305.372 227.508,320.371 232.696,333.249 237.885,343.805 243.073,351.87 248.261,357.319 253.45,360.065 258.638,360.065 263.826,357.319 
 269.015,351.87 274.203,343.805 279.391,333.249 284.58,320.371 289.768,305.372 294.956,288.49 300.145,269.99 305.333,250.165 310.521,229.328 315.71,207.805 
 320.898,185.938 326.086,164.071 331.275,142.549 336.463,121.711 341.651,101.886 346.84,83.3865 352.028,66.5043 357.216,51.5056 362.405,38.6271 367.593,28.0717 
 372.781,20.006 377.97,14.5571 383.158,11.811 388.346,11.811 393.535,14.5571 398.723,20.006 403.911,28.0717 409.1,38.6271 414.288,51.5056 419.477,66.5043 
 424.665,83.3865 429.853,101.886 435.042,121.711 440.23,142.549 445.418,164.071 450.607,185.938 455.795,207.805 460.983,229.328 466.172,250.165 471.36,269.99 
 476.548,288.49 481.737,305.372 486.925,320.371 492.113,333.249 497.302,343.805 502.49,351.87 507.678,357.319 512.867,360.065 518.055,360.065 523.243,357.319 
 528.432,351.87 533.62,343.805 538.808,333.249 543.997,320.371 549.185,305.372 554.373,288.49 559.562,269.99 564.75,250.165 569.938,229.328 575.127,207.805 
 580.315,185.938 
 "/>
<polyline clip-path="url(#clip0002)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 66.6696,185.938 77.0463,185.938 87.4229,185.938 97.7996,185.938 108.176,185.938 118.553,185.938 128.93,185.938 139.306,185.938 149.683,185.938 160.06,185.938 
 170.436,185.938 180.813,185.938 191.19,185.938 201.566,185.938 211.943,185.938 222.32,185.938 232.696,185.938 243.073,185.938 253.45,185.938 263.826,185.938 
 274.203,185.938 284.58,185.938 294.956,185.938 305.333,185.938 315.71,185.938 326.086,185.938 336.463,185.938 346.84,185.938 357.216,185.938 367.593,185.938 
 377.97,185.938 388.346,185.938 398.723,185.938 409.1,185.938 419.477,185.938 429.853,185.938 440.23,185.938 450.607,185.938 460.983,185.938 471.36,185.938 
 481.737,185.938 492.113,185.938 502.49,185.938 512.867,185.938 523.243,185.938 533.62,185.938 543.997,185.938 554.373,185.938 564.75,185.938 575.127,185.938 
 
 "/>
 
 
 
 
 u exact 
 
 
 
 u LF

Finite volume schemes like the Gudonov, Lax-Friedrichs and Engquist-Osher are quite stable with no spurious oscillations or other numerical artifacts. However, the approximation may lead to large errors, particulary at coarse meshes.

An explanation for the large errors is provided in the experimental order of convergence. The observed order of convergence is close to one. This implies that the convergence is slow and errors are reduced very slowly.

In [7]:
#Define No of cells for each iteration
NN = [40,80,160,320,640,1280,2560]
#compute relative errors and experimental order of convergence
function get_error_table(alg, get_problem, NN)
    errors = zeros(Float64,size(NN,1),3)
    for (i,N) in enumerate(NN)
        prob = get_problem(N, 10.0)
        sol = solve(prob, alg;use_threads = true, save_everystep = false, TimeIntegrator = Euler());
        errors[i,1] = N
        errors[i,2] = get_relative_L1_error(u_exact, sol)
    end
    @. errors[2:end,3] = -log(errors[1:(end-1),2]/errors[2:end,2])/log(errors[1:(end-1),1]/errors[2:end,1]);
    return errors
end
# Column heads: No of cells, Rel Error, Convergence order 
errors = get_error_table(LaxFriedrichsAlgorithm(), get_problem, NN)

7×3 Array{Float64,2}:
   40.0  100.0     0.0        
   80.0  100.0     3.7545e-7  
  160.0   99.9403  0.000861413
  320.0   97.534   0.0351609  
  640.0   84.2868  0.210598   
 1280.0   60.3583  0.481756   
 2560.0   37.038   0.704544   

The table shows the relative error in $L^1$ on a sequence of meshes. The percentage relative error is defined by

$$ e_{\Delta x} = 100 \times \frac{\Vert u_{\Delta x} - u_{\mathrm{ref}}\Vert_{L^1}}{\Vert u_{\mathrm{ref}} \Vert_{L^1}} $$
where $u_{\Delta x}$ is the approximate solution computed on a mesh with cell size $\Delta x$ and $u_{\mathrm{ref}}$ is a reference solution to the continuous problem. We also shown the *experimental order of convergence*,

$$ \theta_{\Delta y} = \frac {\log(e_{\Delta x}) - \log(e_{\Delta y})}{\log(\Delta x) - \log(\Delta y)} $$

Now, we test a second order numerical scheme: Lax-Wendroff scheme.

In [8]:
prob = get_problem(50, 10.0)
@time sol2 = solve(prob, LaxWendroffAlgorithm();save_everystep = false, use_threads=true, TimeIntegrator = Euler())
plot(0:0.01:1, [u_exact(x,10.0) for x in 0:0.01:1], lab="u exact")
plot!(sol2, lab="u LW")

  1.823508 seconds (3.31 M allocations: 249.898 MiB, 3.18% gc time)


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 x 
 
 
 u 
 
<polyline clip-path="url(#clip0202)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 61.4812,185.938 66.6696,164.071 71.8579,142.549 77.0463,121.711 82.2346,101.886 87.4229,83.3865 92.6113,66.5043 97.7996,51.5056 102.988,38.6271 108.176,28.0717 
 113.365,20.006 118.553,14.5571 123.741,11.811 128.93,11.811 134.118,14.5571 139.306,20.006 144.495,28.0717 149.683,38.6271 154.871,51.5056 160.06,66.5043 
 165.248,83.3865 170.436,101.886 175.625,121.711 180.813,142.549 186.001,164.071 191.19,185.938 196.378,207.805 201.566,229.328 206.755,250.165 211.943,269.99 
 217.131,288.49 222.32,305.372 227.508,320.371 232.696,333.249 237.885,343.805 243.073,351.87 248.261,357.319 253.45,360.065 258.638,360.065 263.826,357.319 
 269.015,351.87 274.203,343.805 279.391,333.249 284.58,320.371 289.768,305.372 294.956,288.49 300.145,269.99 305.333,250.165 310.521,229.328 315.71,207.805 
 320.898,185.938 326.086,164.071 331.275,142.549 336.463,121.711 341.651,101.886 346.84,83.3865 352.028,66.5043 357.216,51.5056 362.405,38.6271 367.593,28.0717 
 372.781,20.006 377.97,14.5571 383.158,11.811 388.346,11.811 393.535,14.5571 398.723,20.006 403.911,28.0717 409.1,38.6271 414.288,51.5056 419.477,66.5043 
 424.665,83.3865 429.853,101.886 435.042,121.711 440.23,142.549 445.418,164.071 450.607,185.938 455.795,207.805 460.983,229.328 466.172,250.165 471.36,269.99 
 476.548,288.49 481.737,305.372 486.925,320.371 492.113,333.249 497.302,343.805 502.49,351.87 507.678,357.319 512.867,360.065 518.055,360.065 523.243,357.319 
 528.432,351.87 533.62,343.805 538.808,333.249 543.997,320.371 549.185,305.372 554.373,288.49 559.562,269.99 564.75,250.165 569.938,229.328 575.127,207.805 
 580.315,185.938 
 "/>
<polyline clip-path="url(#clip0202)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 66.6696,43.8414 77.0463,30.7689 87.4229,27.4462 97.7996,34.0823 108.176,50.2599 118.553,74.9628 128.93,106.639 139.306,143.297 149.683,182.635 160.06,222.18 
 170.436,259.448 180.813,292.098 191.19,318.077 201.566,335.753 211.943,344.015 222.32,342.346 232.696,330.848 243.073,310.245 253.45,281.832 263.826,247.393 
 274.203,209.093 284.58,169.338 294.956,130.626 305.333,95.3893 315.71,65.8423 326.086,43.8414 336.463,30.7689 346.84,27.4462 357.216,34.0823 367.593,50.2599 
 377.97,74.9628 388.346,106.639 398.723,143.297 409.1,182.635 419.477,222.18 429.853,259.448 440.23,292.098 450.607,318.077 460.983,335.753 471.36,344.015 
 481.737,342.346 492.113,330.848 502.49,310.245 512.867,281.832 523.243,247.393 533.62,209.093 543.997,169.338 554.373,130.626 564.75,95.3893 575.127,65.8423 
 
 "/>
 
 
 
 
 u exact 
 
 
 
 u LW

In [9]:
# Column heads: No of cells, Rel Error, Convergence order 
errors = get_error_table(LaxWendroffAlgorithm(), get_problem, NN)

7×3 Array{Float64,2}:
   40.0  127.138      0.0    
   80.0   37.949      1.74426
  160.0    9.66816    1.97275
  320.0    2.42158    1.99729
  640.0    0.60556    1.99961
 1280.0    0.151398   1.99993
 2560.0    0.0378497  1.99999

# Remarks

* Note that we are using the method of lines to solve each problem. The default time integration algorithm is a second order strong stability preserving Runge-Kutta or SSPRK22 but we use Forward Euler for error estimation. We can change the time integration scheme by using:

```julia
#Solve problem using Lax Friedrichs scheme and Strong Stability Preserving RK33
@time sol = solve(prob, LaxFriedrichsAlgorithm();TimeIntegrator = SSPRK33())
```

* We fixed the CFL to 0.5. Important properties like convergence or TVD can be guaranteed only if an appropriate CFL condition is met

## References
* Siddartha Misha, Numerical Methods for conservation laws and related equations, [https://www2.math.ethz.ch/education/bachelor/lectures/fs2013/math/nhdgl/numcl_notes_HOMEPAGE.pdf]